***1) Install dependencies***

In [ ]:
# Colab cell (code)
# Install required packages
!pip install -q transformers datasets evaluate rouge_score sentencepiece sacrebleu accelerate


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.9 MB/s eta 0:00:00


***2) Load the dataset***

In [ ]:
# Colab cell (code)
from datasets import load_dataset

# load split version 3.0.0 (commonly used for summarization)
dataset = load_dataset("cnn_dailymail", "3.0.0")
# dataset has 'train', 'validation', 'test' with fields: 'article', 'highlights', ...
print(dataset)
print(dataset["train"][0]["article"][:400])
print("summary example:", dataset["train"][0]["highlights"][:200])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his ca
summary example: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been hel


***3) Load model + tokenizer***

In [ ]:
# Colab cell (code)
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# change this to your model choice:
MODEL_NAME = "t5-base"                 # or "t5-small", "facebook/bart-base", "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

***4) Preprocessing function (tokenization)***

In [ ]:
# Colab cell (code)
from datasets import DatasetDict
import numpy as np

max_input_length = 512
max_target_length = 128
source_prefix = "summarize: " if "t5" in MODEL_NAME.lower() else ""

def preprocess_function(examples):
    inputs = [source_prefix + a for a in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")
    # tokenize summaries (targets)
    labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True, padding="max_length")
    # Important: set label pad token id to -100 for PyTorch so it's ignored in loss
    model_inputs["labels"] = [
        [(lbl if lbl != tokenizer.pad_token_id else -100) for lbl in l] for l in labels["input_ids"]
    ]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
# Inspect shapes
print(tokenized_datasets["train"].features)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

{'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8')), 'labels': List(Value('int64'))}


In [ ]:
# For quick debug runs
small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_valid = tokenizedhbd_datasets["validation"].shuffle(seed=42).select(range(500))


In [ ]:
evaluation_strategy="steps",


In [ ]:
evaluate_during_training=True,


In [ ]:

import transformers
print(transformers.__version__)


4.57.1


In [ ]:
# Colab cell (code) — diagnostic
import transformers, sys, inspect
print("transformers.__version__ =", transformers.__version__)
from transformers import Seq2SeqTrainingArguments
print("Seq2SeqTrainingArguments object:", Seq2SeqTrainingArguments)
print("Defined in module:", Seq2SeqTrainingArguments.__module__)
print("Constructor signature:")
try:
    print(inspect.signature(Seq2SeqTrainingArguments.__init__))
except Exception as e:
    print("Could not get signature:", e)
# show file location
import importlib, os
mod = importlib.import_module(Seq2SeqTrainingArguments.__module__)
print("module file:", getattr(mod, "__file__", "built-in or not a file"))


transformers.__version__ = 4.57.1
Seq2SeqTrainingArguments object: <class 'transformers.training_args_seq2seq.Seq2SeqTrainingArguments'>
Defined in module: transformers.training_args_seq2seq
Constructor signature:
(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0,

***5)Training arguments + DataCollator***

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

output_dir = "/content/outputs/t5-summarization"  # change if you mounted drive

batch_size = 4   # change based on GPU memory. Use 8/16 for bigger GPUs.
args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",  # Changed from evaluation_strategy to eval_strategy
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=200,
    save_steps=1000,
    eval_steps=1000,
    num_train_epochs=1,       # set to 3+ for real runs
    save_total_limit=2,
    fp16=True,                # set True if using Colab GPU (if supported)
    gradient_accumulation_steps=8,  # if your batch_size is small, accumulate gradients
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

***6)Compute metrics (ROUGE)***

In [ ]:
# Colab cell (code)
import evaluate
import numpy as np

rouge = evaluate.load("rouge")

# helper to decode predictions & labels
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # replace -100 in labels (the Trainer will already handle -100 if you pass labels from tokenized dataset)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # rouge returns dict with rouge1/2/... as floats; convert to percentages
    result = {k: round(v * 100, 4) for k, v in result.items()}
    # add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


***7)Create trainer and start training***

In [ ]:
# Colab cell (code)
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"].select(range(20000)),   # adjust or remove select for full dataset
    eval_dataset=tokenized_datasets["validation"].select(range(2000)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


/tmp/ipython-input-1205824171.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: f223068 (f223068-fast-universuty) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


TrainOutput(global_step=625, training_loss=1.6054349670410155, metrics={'train_runtime': 1592.1522, 'train_samples_per_second': 12.562, 'train_steps_per_second': 0.393, 'total_flos': 1.21791578112e+16, 'train_loss': 1.6054349670410155, 'epoch': 1.0})

***8)Save trained model***

In [ ]:
# Colab cell (code)
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print("Saved to", output_dir)


Saved to /content/outputs/t5-summarization


***9) Load model for generation & helper function***

In [ ]:
# Colab cell (code)
# load model & tokenizer if needed
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

def generate_summary(text, max_length=128, num_beams=4):
    input_text = (source_prefix + text) if "t5" in MODEL_NAME.lower() else text
    inputs = tokenizer([input_text], max_length=max_input_length, truncation=True, return_tensors="pt")
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    # generate
    outputs = model.generate(input_ids=input_ids,
                             attention_mask=attention_mask,
                             max_length=max_length,
                             num_beams=num_beams,
                             length_penalty=2.0,
                             early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate on a small slice of test set
test_slice = dataset["test"].select(range(100))  # pick 100 examples for metric compute speed
preds = []
refs = []
for ex in test_slice:
    pred = generate_summary(ex["article"])
    preds.append(pred)
    refs.append(ex["highlights"])

# compute rouge
r = evaluate.load("rouge")
res = r.compute(predictions=preds, references=refs, use_stemmer=True)
res = {k: round(v*100,4) for k,v in res.items()}
print("ROUGE on 100 test examples:", res)

# Print some qualitative examples
for i in range(5):
    print(f"\n=== EXAMPLE {i+1} ===")
    print("Article (truncated):", test_slice[i]["article"][:500])
    print("\nReference Summary:", test_slice[i]["highlights"])
    print("\nGenerated Summary:", preds[i])


KeyboardInterrupt: 

**Evaluate on test slice (automatic ROUGE)**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your fine-tuned model
model_dir = "/content/outputs/t5-summarization"  # adjust if different
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Sample article (you can replace with any text)
article_text = """
NASA's Perseverance rover has successfully collected its first samples of Martian rock,
marking a historic step in humanity's exploration of Mars. The samples will be returned
to Earth by a future mission, allowing scientists to study the planet's geology and
look for signs of ancient life. The rover has been exploring the Jezero Crater,
an ancient river delta, since its landing in February 2021.
"""

# Tokenize the article
inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=512)

# Generate summary
summary_ids = model.generate(
    **inputs,
    max_length=100,    # max length of summary
    num_beams=4,       # beam search for better quality
    early_stopping=True
)

# Decode to text
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Original Article:\n", article_text)
print("\nGenerated Summary:\n", summary)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Original Article:
 
NASA's Perseverance rover has successfully collected its first samples of Martian rock, 
marking a historic step in humanity's exploration of Mars. The samples will be returned 
to Earth by a future mission, allowing scientists to study the planet's geology and 
look for signs of ancient life. The rover has been exploring the Jezero Crater, 
an ancient river delta, since its landing in February 2021.


Generated Summary:
 NASA's Perseverance rover has successfully collected its first samples of Martian rock. The samples will be returned to Earth by a future mission, allowing scientists to study the planet's geology and look for signs of ancient life.


***Print qualitative examples***

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path to your trained model in Colab
model_dir = "/content/outputs/t5-summarization"

# Load model and tokenizer (if not already loaded)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Save the model and tokenizer (weights + vocab + config)
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)


('/content/outputs/t5-summarization/tokenizer_config.json',
 '/content/outputs/t5-summarization/special_tokens_map.json',
 '/content/outputs/t5-summarization/spiece.model',
 '/content/outputs/t5-summarization/added_tokens.json',
 '/content/outputs/t5-summarization/tokenizer.json')

In [ ]:
from google.colab import files
import shutil

# Zip the entire folder
shutil.make_archive("t5_summarization_model", 'zip', model_dir)

# Download zip fileI
files.download("t5_summarization_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your fine-tuned model
model_dir = "/content/outputs/t5-summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Random example article
article_text = """
Apple has announced the launch of its new iPhone 15 series, featuring improved cameras,
a faster A17 processor, and longer battery life. The new models also include USB-C charging,
a first for iPhones, and come in a variety of colors. Pre-orders begin this week with
shipping expected next month. Analysts predict strong sales due to high consumer demand
and the popularity of the upgraded camera system.
"""

# Tokenize the article
inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=512)

# Generate summary
summary_ids = model.generate(
    **inputs,
    max_length=100,
    num_beams=4,
    early_stopping=True
)
predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Display result
print("=== Random Test Example ===")
print("Predicted Summary:\n", predicted_summary)


=== Random Test Example ===
Predicted Summary:
 iPhone 15 series. The new models also include USB-C charging, a first for iPhones, and come in a variety of colors. Pre-orders begin this week with shipping expected next month.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your fine-tuned model
model_dir = "/content/outputs/t5-summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

# Another example article (different topic)
article_text = """
The United Nations has issued a warning about the increasing severity of climate change
effects worldwide. Rising global temperatures are causing more frequent and intense
wildfires, floods, and hurricanes. Governments are urged to take urgent action
to reduce carbon emissions and invest in sustainable energy solutions to prevent
catastrophic environmental consequences.
"""

# Tokenize the article
inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=512)

# Generate summary
summary_ids = model.generate(
    **inputs,
    max_length=100,
    num_beams=4,
    early_stopping=True
)
predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Display result
print("=== Climate Change Article Example ===")
print("Predicted Summary:\n", predicted_summary)


=== Climate Change Article Example ===
Predicted Summary:
 The United Nations has issued a warning about the increasing severity of climate change effects worldwide. Rising global temperatures are causing more frequent and intense wildfires, floods, and hurricanes.
